In [ ]:
import subprocess
import shlex
import re

import numpy as np

from IPython.display import display, Markdown
from tqdm import tqdm

In [ ]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

# Dieharder - A Random Number Test Suite

In [ ]:
def test_randomness_dieharder(
    mode, seeds=[42], args=[]
):
    outputs = []
    
    for seed in tqdm(seeds):
        generator = subprocess.Popen(shlex.split(
            f"cargo run --release --quiet -- --seed {seed} {CC_PATTERN.sub('-', mode).lower()}"
        ), stdout=subprocess.PIPE)
        dieharder = subprocess.run(
            shlex.split(f"dieharder -g 200") + args, stdin=generator.stdout,
            check=True, capture_output=True, text=True
        )

        outputs.append(dieharder.stdout)
        
    most_suspect = None
    
    for output in outputs:
        failures = output.count("FAILED")
        weaks = output.count("WEAK")
        
        suspicion = failures * 10 + weaks
        
        if (most_suspect is None) or (suspicion > most_suspect[0]):
            most_suspect = (suspicion, output)

    if "FAILED" in most_suspect[1]:
        display(Markdown(f"## <span style='color:red'><u>{mode}</u></span>"))
    elif "WEAK" in most_suspect[1]:
        display(Markdown(f"## <span style='color:orange'>*{mode}*</span>"))
    else:
        display(Markdown(f"## <span style='color:green'>{mode}</span>"))
        
    display(Markdown("#### Test output (most suspect):"))
    display(Markdown(">" + most_suspect[1].replace('\n', '\n>')))

    display(Markdown("#### Parameters:"))
    display(Markdown(f"* seeds: {seeds}"))

In [ ]:
for mode in ["Monolithic", "Independent", "IndependentSimulation", "IndependentSimulationNoDispersal"]:
    seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=1)
    
    test_randomness_dieharder(mode, seeds=seeds, args=["-a"])